# Algoritmo que permite resolver problemas QUBO densos empleando Tensor Network

En este notebook vamos a crear e implementar un algoritmo en tensor networks basado en el método de las señales y evolución en tiempo imaginario para resolver problemas QUBO. Utiliza un método de eliminación de capas de evolución para reducir la complejidad computacional.

Versiones:

- alpha 0: Implementación básica.
- alpha 1: Añade el método de eliminación de capas.
- alpha 2: Versión que va comprimiendo el MPS de derecha a izquierda
- alpha 3: version que aplica el problema QUBO a los vecinos cercanos, la idea de esta versión es aplicar matrices recursivamente, además de intentar aprovechar los cálculos intermedios
        3b: version 2
        3b2: version 2 refactorizada
        

In [2]:
# Librerias
import numpy as np

from quantum_sim.main.general_functions import dinariy_list
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_solvers import qubo_dimod_solver, recocido_simulado, qubo_solver_rs, random_qubo_solver
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_auxiliar_functions import matrix_QUBO_to_dict, evaluar_qubo, generar_matriz_qubo
from quantum_sim.main.guardar_experimentos import plot_function
from itertools import product
from time import time
from math import log
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'quantum_sim'

---
# Funciones de la tensor network

## Definimos los nodos de la Tensor Network

In [2]:
from itertools import combinations
from operator import index

from numpy import dtype
from sklearn import neighbors


def node_0(Q_matrix_0: float, dits: int, tau: float):
    tensor = np.zeros((dits, dits))
    for index_a in range(dits):
        tensor[index_a, index_a] = np.exp(-tau* Q_matrix_0 * index_a)
    return tensor

def node_grow(Q_matrix_row: np.array, dits: int, num_neight_updown, tau: float):
    size_1 = dits**num_neight_updown
    size_2 = size_1 * dits
    tensor = np.zeros((size_1, size_2))
    dit_list = list(range(dits))
    combinations_up = product(dit_list, repeat = num_neight_updown)
    for element in combinations_up:
        for index_last in range(dits):
            index_up = 0
            for aux in range(num_neight_updown):
                index_up += dits**aux*element[aux]
            index_down = index_up + dits**(aux+1)*index_last
            full_element = []
            for aux in element:
                full_element.append(aux)
            full_element.append(index_last)
            tensor[index_up, index_down] = 1
            for aux in range(len(full_element)):
                tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[aux]*full_element[-1]*full_element[aux])
    return tensor

def node_intermediate(Q_matrix_row: np.array, dits: int, num_neight_updown, tau: float):
    size_1 = dits**num_neight_updown
    tensor = np.zeros((size_1, size_1))

    dit_list = list(range(dits))
    combinations_up = product(dit_list, repeat = num_neight_updown)
    for element in combinations_up:
        for index_last in range(dits):
            full_element = []
            index_up = 0
            for aux in range(num_neight_updown):
                index_up += dits**aux*element[aux]   
            for aux in range(1,len(element)):
                full_element.append(element[aux])
            full_element.append(index_last)
            index_down = 0
            for aux in range(num_neight_updown):
                index_down += dits**aux*full_element[aux]   
            full_element.insert(0,element[0])
            tensor[index_up, index_down] = 1
            for aux in range(len(full_element)):
                tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[aux]*full_element[-1]*full_element[aux])
   
    return tensor
                    

def last_tensor(Q_matrix_row: np.array, dits: int, tau:float):
    n_neighbors = (len(Q_matrix_row)-1)
    tensor_size = int(dits**(n_neighbors))
    tensor = np.zeros((tensor_size))
    dit_list = list(range(dits))
    combinations_up = product(dit_list, repeat = n_neighbors)
    for element in combinations_up:
        index_up = 0
        for aux in range(n_neighbors):
            index_up += dits**aux*element[aux]
        for index_last in range(dits):
            full_element = []
            tensor_aux =1
            for aux in element:
                full_element.append(aux)
            full_element.append(index_last)
            for el in range(len(full_element)):
                tensor_aux *= np.exp(-tau*Q_matrix_row[el]*full_element[el]*full_element[-1])
            tensor[index_up] += tensor_aux

    return tensor



def new_inital_tensor(Q_matrix_row, dits: int, size_2, solution, n_neigh, tau:float):

    size_1 = dits
    tensor = np.zeros((size_1, size_2))

    n = len(solution)+1

    dit_list = list(range(dits))
    solution = tuple(solution)
    combinations_up = product(dit_list, repeat = n)
    

    index_down = 0
    for aux in range(len(solution)):
        index_down += dits**aux*solution[aux]


    for element in combinations_up:
        
        if element[:-1] == solution:
  
            index_down_aux = index_down + dits**(n-1)*element[-1]

            tensor[element[-1], index_down_aux] = 1
             
            for el in range(len(element)):          
                tensor[element[-1], index_down_aux] *= np.exp(-tau*Q_matrix_row[el]*element[el]*element[-1])


    return tensor



## Generación de la Tensor Network

In [3]:
def tensor_network_generator(Q_matrix:np.array, dits:float, n_neighbors:int, tau: float):
    """   
    Args:

    Return:

    """

    n_variables  = len(Q_matrix[0])
    intermediate_tensors = []
    # generation of the first node
    tensor = node_0(Q_matrix[0][0], dits, tau)
    intermediate_tensors.append(tensor)
    
    # Generation of the intermediate nodes
    for variable in range(1, n_variables-1):
        
        if variable < n_neighbors:
            Q_matrix_row_input = Q_matrix[variable][max(0, variable-n_neighbors-1):variable+1]
            tensor = node_grow(Q_matrix_row_input, dits, variable, tau)
            intermediate_tensors.append(tensor)
        else:
            Q_matrix_row_input = Q_matrix[variable][variable-n_neighbors :variable+1]
            tensor = node_intermediate(Q_matrix_row_input, dits, n_neighbors, tau)
            intermediate_tensors.append(tensor)

            
    Q_matrix_row_input = Q_matrix[variable + 1][variable + 1-n_neighbors:variable+2]
    tensor = last_tensor(Q_matrix_row_input, dits, tau)
    intermediate_tensors.append(tensor)

    return  intermediate_tensors

## Conexión y contracción de toda la Tensor Network

In [4]:
def tensor_network_contraction(tensor_list: list):
    n_tensores = len(tensor_list)
    intermediate_tensors = []
    tensor = tensor_list[-1]
    intermediate_tensors.append(tensor)
    for index_ in range(n_tensores -1, 0, -1):
        current_tensor = tensor_list[index_ -1]
        tensor = current_tensor @ tensor
        tensor /= np.linalg.norm(tensor)
        intermediate_tensors.append(tensor)
        
    intermediate_tensors.reverse()
    return tensor, intermediate_tensors
    

---
# Función general

Esta es la función que se encarga del proceso general. Se encarga del proceso de minimización resolviendo iterativamente cada una de las variables. Su proceso consiste en la creación de la tensor network, su contracción y la determinación de la variable a partir del vector resultante.

In [5]:
sol = np.zeros((2))
print(len(sol))

2


In [6]:
from unittest import result

from quantum_sim.main.general_functions import base_a_decimal


def qubo_solver(Q_matrix:np.array, tau:float, dits: int, n_neighbors: int)->np.array:
    n_variables = Q_matrix.shape[0]
    #print(Q_matrix)
    original_Q_matrix = Q_matrix.copy()
    solution = np.zeros(n_variables, dtype = int)
    tensor_network = tensor_network_generator(Q_matrix, dits, n_neighbors, tau)

    result_contraction, intermediate_tensors = tensor_network_contraction(tensor_network)

    solution[0] = np.argmax(abs(result_contraction))

    Q_matrix[n_neighbors -1 , n_neighbors -1] += Q_matrix[n_neighbors -1, 0] * solution[0]

    for node in range(1, n_variables-1):

        if node < n_neighbors:
            Q_matrix_row = Q_matrix[node][max(0, node-n_neighbors-1):node+1]
            sol_aux = solution[max(0, node-n_neighbors-1):node]
        else:
            Q_matrix_row = Q_matrix[node][node-n_neighbors + 1:node+1]
            sol_aux = solution[node-n_neighbors+1:node]

        new_tensor = new_inital_tensor(Q_matrix_row, dits, intermediate_tensors[2].shape[0], sol_aux, n_neighbors, tau)

        solution[node] = np.argmax(abs(new_tensor @ intermediate_tensors[2]))
        
        print(new_tensor @ intermediate_tensors[2])
        intermediate_tensors.pop(0)
        # if n_neighbors -1 + node < n_variables:
        #     Q_matrix[n_neighbors -1 + node, n_neighbors -1 + node] += Q_matrix[n_neighbors -1, node] * solution[node]
        for i in range(node + 2,len(Q_matrix)):  # Iteramos sobre las filas
            Q_matrix[i, i] += Q_matrix[i, node] * solution[node]



    cost1 = evaluar_qubo(original_Q_matrix, solution)
    solution[-1] = 1
    cost2 = evaluar_qubo(original_Q_matrix, solution)
    if cost1 < cost2:
        solution[-1] = 0





    return solution


---
# Pruebas

In [7]:
import math
print(math.e**(-100 *(0.26183226+0.4842719)))

3.954649869793464e-33


In [13]:
n_variables = 10
n_vecinos = 4
dits = 3
tau = 100
np.random.seed(53)
# Generamos el caso

Q_matrix = generar_matriz_qubo(n_variables, n_vecinos+1)
Q_matrix/= np.linalg.norm(Q_matrix)
Q_matrix_dict = matrix_QUBO_to_dict(Q_matrix)

Q_matrix_copy = Q_matrix.copy()
# Inicial RS
x_inicial = np.random.randint(2, size=n_variables)



# Dimod
#solution_dimod = qubo_dimod_solver(Q_matrix_dict, "neal")
#print('Coste de dimod iter: ', evaluar_qubo(Q_matrix, solution_dimod))

#best_c = random_qubo_solver(Q_matrix)
#print('Coste de Random: ', evaluar_qubo(Q_matrix, best_c))

# TN
inicio = time()
solution = qubo_solver(Q_matrix_copy, tau, dits, n_vecinos)
print("tiempo", time()-inicio)
print('Solution TN:     ', str(solution))
print('Coste de TN:     ', evaluar_qubo(Q_matrix, solution))
'''
# RS
mejor_solucion, mejor_valor = recocido_simulado_qudo(Q_matrix, [0, 1, 2], 100, 0.95, 1000)
print("Mejor solución encontrada:", mejor_solucion)
print("Mejor valor objetivo:", evaluar_qubo(Q_matrix, mejor_solucion))
'''


[4.15596964e-04 1.20480742e-24 3.22735374e-57]
[4.15596964e-04 5.45829642e-29 2.12110162e-52]
[3.14270478e-83 7.02713420e-54 4.27628309e-48]
[1.00000000e+00 1.42813723e-36 8.17276352e-86]
[1.37759508e-057 6.45230247e+002 3.63303742e+101]
[5.85443992e-007 4.11370169e+050 4.94247484e+140]
[1.08262165e-003 8.08924310e-071 4.01013534e-186]
[1072.65540015 2456.35516554    5.89955773]
tiempo 0.018477678298950195
Solution TN:      [0 0 0 2 0 2 2 0 1 0]
Coste de TN:      -1.0307780170199132


'\n# RS\nmejor_solucion, mejor_valor = recocido_simulado_qudo(Q_matrix, [0, 1, 2], 100, 0.95, 1000)\nprint("Mejor solución encontrada:", mejor_solucion)\nprint("Mejor valor objetivo:", evaluar_qubo(Q_matrix, mejor_solucion))\n'